In [ ]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [ ]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [ ]:
# vocab
vocab = {'<UNK>':0}

In [ ]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [ ]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
len(vocab)

324

In [ ]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [ ]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [ ]:
dataset = QADataset(df, vocab)

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[10, 96,  3, 97]]) tensor([98])
tensor([[ 10, 308,   3, 309, 310]]) tensor([311])
tensor([[ 10,  11, 157, 158, 159]]) tensor([160])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([61])
tensor([[ 10, 140,   3, 141, 142,  12, 143,  83,   3, 144]]) tensor([145])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([205])
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]]) tensor([23])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([295])
tensor([[10, 75, 76]]) tensor([77])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([194])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([268])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([16])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([276])
tensor([[  1,   2,   3, 

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [ ]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [ ]:
learning_rate = 0.001
epochs = 20

In [ ]:
model = SimpleRNN(len(vocab))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 526.661325
Epoch: 2, Loss: 449.646401
Epoch: 3, Loss: 373.457404
Epoch: 4, Loss: 319.754306
Epoch: 5, Loss: 270.640235
Epoch: 6, Loss: 223.933745
Epoch: 7, Loss: 181.511743
Epoch: 8, Loss: 143.123770
Epoch: 9, Loss: 111.379831
Epoch: 10, Loss: 86.361953
Epoch: 11, Loss: 67.281734
Epoch: 12, Loss: 53.119437
Epoch: 13, Loss: 42.449729
Epoch: 14, Loss: 34.707759
Epoch: 15, Loss: 28.854543
Epoch: 16, Loss: 24.150912
Epoch: 17, Loss: 20.600887
Epoch: 18, Loss: 17.434267
Epoch: 19, Loss: 15.025007
Epoch: 20, Loss: 12.968341


In [ ]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [ ]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [ ]:
list(vocab.keys())[7]

'paris'